In [1]:
!pip install tensorboard

In [24]:
import sys
sys.path.append('..')


import yaml
from net_architectures.lstm import *
from net_architectures.mlp import *
from gensim.models import Word2Vec
import pandas as pd
import random
#tensorboard --logdir=/home/diego/Documentos/MTG_drafter/17_lands/runs --verbosity 0

random.seed(42)

# Verificar se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
word_to_vec_model = Word2Vec.load("card2vec.model")
vocab = list(word_to_vec_model.wv.key_to_index.keys())

In [26]:
import ast
df_train = pd.read_csv('dataset/train_set.csv')
# Convertendo as strings para listas
df_train['pool'] = df_train['pool'].apply(ast.literal_eval)
# drop rnan pick
df_train = df_train.dropna(subset=['pick'])

df_train['label'] = df_train['pick'].apply(lambda x: vocab.index(x))


In [27]:
from dataset_class import *

# Crie o dataset e o DataLoader
dataset = MagicDataset(df_train, word_to_vec_model, vocab)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# Itere sobre as primeiras 45 entradas do DataLoader
for i, (inputs, labels, mascara) in enumerate(dataloader):
    if i < 45:
        print(f"--- Entrada {i+1} ---")
        
        # Converta os embeddings para nomes
        input_names = [MagicDataset.embedding_para_nome(embedding.numpy(), word_to_vec_model) for embedding in inputs.squeeze(0)]
        
        # Converta o índice da label para o nome usando a função indice_para_nome
        label_name = vocab[labels.item()]
        print(labels.item())
        print("Inputs:", input_names)
        print("Label:", label_name)
        print("Máscara:", mascara)
        print("\n")
        break
    else:
        break


--- Entrada 1 ---
232
Inputs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Label: Archon of the Wild Rose
Máscara: tensor([[1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [28]:
# Instanciar o modelo LSTM
from torch import nn
from net_architectures.mlp import *
#model = LSTMModel()
model = FeedForwardModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [30]:
#filter rank == diamond
df_train = df_train[df_train['rank'] == 'diamond']

In [31]:
import os
import torch
from torch.utils.tensorboard import SummaryWriter
from dataset_class import *
from torch.utils.data import DataLoader

# Defina os hiperparâmetros
num_epochs = 50
batch_size = 64
model_name = 'model_1'

# Configurar o TensorBoard SummaryWriter para escrever logs na pasta 'runs'
log_dir = './runs/'+model_name+'/'
writer = SummaryWriter(log_dir=log_dir)

# Defina um diretório para salvar os checkpoints
checkpoint_dir = 'checkpoints'

# Certifique-se de que o diretório exista (ou crie-o)
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Criar o dataset e o DataLoader
dataset = MagicDataset(df_train, word_to_vec_model, vocab)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Mover o modelo para a GPU
model.to(device)

# Ciclo de treinamento
for epoch in range(num_epochs):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for i, (inputs, labels, mascara) in enumerate(dataloader):
        # Mover as entradas e rótulos para a GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        mascara = mascara.to(device)

        # Zerar os gradientes
        optimizer.zero_grad()

        # Forward
        outputs = model(inputs, mascara)
        loss = criterion(outputs, labels)
        
        # Acumular a perda
        total_loss += loss.item()

        # Backward
        loss.backward()
        optimizer.step()

        # Calcular acertos
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Calcular e imprimir a perda média e a acurácia da época
    average_loss = total_loss / len(dataloader)
    epoch_accuracy = correct_predictions / total_samples
    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}, Epoch Acuracy: {epoch_accuracy:.4f}")

    # Registrar no TensorBoard
    writer.add_scalar('Epoch Accuracy', epoch_accuracy, epoch)
    writer.add_scalar('Epoch Loss', average_loss, epoch)

    # Salvar o estado do modelo a cada 5 épocas
    if (epoch + 1) % 5 == 0:
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': average_loss
        }
        checkpoint_path = os.path.join(checkpoint_dir, f'{model_name}_checkpoint_epoch_{epoch + 1}.pt')
        torch.save(checkpoint, checkpoint_path)

# Fechar o writer após terminar o treinamento
writer.close()


Epoch [1/50], Average Loss: 1.5206, Epoch Acuracy: 0.4534
Epoch [2/50], Average Loss: 1.4169, Epoch Acuracy: 0.4694
Epoch [3/50], Average Loss: 1.4032, Epoch Acuracy: 0.4726
Epoch [4/50], Average Loss: 1.3948, Epoch Acuracy: 0.4755
Epoch [5/50], Average Loss: 1.3906, Epoch Acuracy: 0.4772
Epoch [6/50], Average Loss: 1.3871, Epoch Acuracy: 0.4775
Epoch [7/50], Average Loss: 1.3839, Epoch Acuracy: 0.4779
Epoch [8/50], Average Loss: 1.3820, Epoch Acuracy: 0.4793
Epoch [9/50], Average Loss: 1.3801, Epoch Acuracy: 0.4795
Epoch [10/50], Average Loss: 1.3788, Epoch Acuracy: 0.4808
Epoch [11/50], Average Loss: 1.3777, Epoch Acuracy: 0.4804
Epoch [12/50], Average Loss: 1.3764, Epoch Acuracy: 0.4812
Epoch [13/50], Average Loss: 1.3755, Epoch Acuracy: 0.4812
Epoch [14/50], Average Loss: 1.3746, Epoch Acuracy: 0.4813
Epoch [15/50], Average Loss: 1.3740, Epoch Acuracy: 0.4815
Epoch [16/50], Average Loss: 1.3733, Epoch Acuracy: 0.4818
Epoch [17/50], Average Loss: 1.3728, Epoch Acuracy: 0.4817
Epoch 

: 